# Example notebook

This is page exists to show you how to add additional notebooks to the book. Notebooks are used to display functional code, outputs and text together in the same document.

## Example code - Calculating distances between centroids

Load and join up the example data:

In [1]:
import geopandas as gpd
import pandas as pd

# Load and join GMCA housing, industrial and office supply data

housing_supply_gdf = gpd.read_file("data/gmca_data/2024 GM Housing Land Supply GIS.shp")
industrial_supply_gdf = gpd.read_file("data/gmca_data/2024 GM Industrial-warehousing Land Supply GIS.shp")
offices_supply_gdf = gpd.read_file("data/gmca_data/2024 GM Offices Land Supply GIS.shp")

total_supply_gdf = pd.concat(
    [housing_supply_gdf, industrial_supply_gdf, offices_supply_gdf]
)

# Load and tidy GMEU Sites of Biological Importance data

sbi_gdf = gpd.read_file("data/gmeu_data/gm_sbi.shp")

sbi_gdf["Category"] = "Site of Biological Importance"

sbi_gdf = sbi_gdf.rename(columns = {"district": "LAName", "site_nam": "SiteRef"})

# Join GMCA and GMEU data

full_data_gdf = pd.concat(
    [total_supply_gdf, sbi_gdf[["SiteRef", "LAName", "Category", "geometry"]]]
)

We can use GeoPandas to get the centroids of all our data:

In [2]:
full_data_gdf["centroid"] = full_data_gdf.centroid

Now we can extract the location of a random SBI and a random proposed development site:

In [3]:
random_sbi = full_data_gdf[
    full_data_gdf["Category"] == "Site of Biological Importance"
    ].sample(1)

random_sbi = random_sbi["centroid"].iloc[0]

print(random_sbi)

POINT (394262.146900225 393853.71414160775)


In [4]:
random_housing = full_data_gdf[
    full_data_gdf["Category"] == "Housing"
    ].sample(1)

random_housing = random_housing["centroid"].iloc[0]

print(random_housing)

POINT (380885.7177838871 413390.49425104103)


The distance between two Point objects can be calculated using Shapely:

In [5]:
import shapely

print(
    round(shapely.distance(random_housing, random_sbi), 2)
)

23677.3


In [6]:
full_data_gdf.sample(10)

,Category,LAName,SiteRef,geometry,centroid
2655,Housing,Trafford,1838,"POLYGON ((378599.23 392777.879, 378611.09 3927...",POINT (378604.08 392772.279)
2329,Housing,Tameside,H-DENTNE-004,"POLYGON ((392471.35 394974.3, 392495.1 394982....",POINT (392502.167 394968.777)
3349,Housing,Bolton,HLA-164,"POLYGON ((363091.45 411165.05, 363095.05 41117...",POINT (363109.934 411166.867)
1376,Housing,Oldham,HLA3147,"POLYGON ((400314.122 407524.903, 400314.238 40...",POINT (400278.472 407471.859)
773,Housing,Manchester,133426/FO/2022,"POLYGON ((383458.93 390868.729, 383473.032 390...",POINT (383481.046 390857.056)
419,Housing,Manchester,Wood2300,"POLYGON ((382349.771 387081.344, 382351.718 38...",POINT (382390.374 387078.525)
127,Industrial/warehousing,Oldham,OLD0092,"POLYGON ((393277.05 405511.35, 393293.15 40549...",POINT (393277.68 405495.499)
1328,Housing,Oldham,SHA1599,"POLYGON ((395807.726 405508.98, 395893.361 405...",POINT (395843.397 405468.872)
2392,Housing,Tameside,H-MOSSLE-139,"POLYGON ((397656.047 402742.42, 397677.021 402...",POINT (397635.73 402707.121)
144,Offices,Oldham,OLD0155,"POLYGON Z ((393289.75 405144.6 0, 393289.15 40...",POINT (393301.31 405156.97)


In [15]:
import geopandas as gpd

boundaries = gpd.read_file("data/boundaries/LAD_DEC_24_UK_BUC.shp")

pd.merge(full_data_gdf, boundaries, left_on = "LAName", right_on = "LAD24NM", how = "left")

,Category,LAName,SiteRef,geometry_x,centroid,LAD24CD,LAD24NM,LAD24NMW,BNG_E,BNG_N,LONG,LAT,GlobalID,geometry_y
0,Housing,Bury,HL/0414/00,"POLYGON ((379112.8 410310.51, 379117.56 410306...",POINT (379114.012 410213.116),E08000002,Bury,None,379658,410767,-2.30880,53.5931,9b925fe0-4d83-431f-83c3-434ba20a3183,"POLYGON ((382118.504 413135.864, 383878.498 41..."
1,Housing,Bury,HL/1172/00,"POLYGON ((377973.95 414475.81, 377979.24 41446...",POINT (377947.835 414384.462),E08000002,Bury,None,379658,410767,-2.30880,53.5931,9b925fe0-4d83-431f-83c3-434ba20a3183,"POLYGON ((382118.504 413135.864, 383878.498 41..."
2,Housing,Bury,HL/1981/00,"POLYGON ((380839 413359.78, 380838.6 413361.95...",POINT (380885.718 413390.494),E08000002,Bury,None,379658,410767,-2.30880,53.5931,9b925fe0-4d83-431f-83c3-434ba20a3183,"POLYGON ((382118.504 413135.864, 383878.498 41..."
3,Housing,Bury,HL/2004/00,"POLYGON ((378834 407567.1, 378840 407566.81, 3...",POINT (378852.408 407559.6),E08000002,Bury,None,379658,410767,-2.30880,53.5931,9b925fe0-4d83-431f-83c3-434ba20a3183,"POLYGON ((382118.504 413135.864, 383878.498 41..."
4,Housing,Bury,HL/2127/00,"POLYGON ((379113.7 410329.59, 379114.66 410328...",POINT (379117.39 410313.26),E08000002,Bury,None,379658,410767,-2.30880,53.5931,9b925fe0-4d83-431f-83c3-434ba20a3183,"POLYGON ((382118.504 413135.864, 383878.498 41..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4888,Site of Biological Importance,Tameside,Mill Race & Pasture at Haughton Dale,"MULTIPOLYGON (((393687.128 393384.895, 393682....",POINT (393841.341 393512.227),E08000008,Tameside,None,394987,397995,-2.07700,53.4787,d32aeedd-2000-4d80-b729-7484ebf85e64,"POLYGON ((402525.503 401459.003, 400905.596 39..."
4889,Site of Biological Importance,Wigan,Three Sisters,"POLYGON ((357928.939 401185.293, 357957.618 40...",POINT (358367.769 401012.636),E08000010,Wigan,None,362139,402126,-2.57242,53.5145,80163100-345f-4a19-9eae-09f2bda28a68,"POLYGON ((358668.063 410971.684, 361851.801 40..."
4890,Site of Biological Importance,Manchester,Nan Nook Wood,"POLYGON ((381158.403 390125.048, 381156.853 39...",POINT (380888.856 390037.976),E08000003,Manchester,None,384591,397063,-2.23360,53.4701,6a898ce9-19ca-4131-bdb4-6d5d85051598,"POLYGON ((384375.197 404714.801, 385603.299 40..."
4891,Site of Biological Importance,Manchester,Big Wood,"MULTIPOLYGON (((383512.797 386267.952, 383512....",POINT (383309.435 386406.658),E08000003,Manchester,None,384591,397063,-2.23360,53.4701,6a898ce9-19ca-4131-bdb4-6d5d85051598,"POLYGON ((384375.197 404714.801, 385603.299 40..."


In [16]:
joined_one = pd.merge(full_data_gdf, boundaries, left_on = "LAName", right_on = "LAD24NM", how = "left")

In [19]:
joined_one

,Category,LAName,SiteRef,geometry_x,centroid,LAD24CD,LAD24NM,LAD24NMW,BNG_E,BNG_N,LONG,LAT,GlobalID,geometry_y
0,Housing,Bury,HL/0414/00,"POLYGON ((379112.8 410310.51, 379117.56 410306...",POINT (379114.012 410213.116),E08000002,Bury,None,379658,410767,-2.30880,53.5931,9b925fe0-4d83-431f-83c3-434ba20a3183,"POLYGON ((382118.504 413135.864, 383878.498 41..."
1,Housing,Bury,HL/1172/00,"POLYGON ((377973.95 414475.81, 377979.24 41446...",POINT (377947.835 414384.462),E08000002,Bury,None,379658,410767,-2.30880,53.5931,9b925fe0-4d83-431f-83c3-434ba20a3183,"POLYGON ((382118.504 413135.864, 383878.498 41..."
2,Housing,Bury,HL/1981/00,"POLYGON ((380839 413359.78, 380838.6 413361.95...",POINT (380885.718 413390.494),E08000002,Bury,None,379658,410767,-2.30880,53.5931,9b925fe0-4d83-431f-83c3-434ba20a3183,"POLYGON ((382118.504 413135.864, 383878.498 41..."
3,Housing,Bury,HL/2004/00,"POLYGON ((378834 407567.1, 378840 407566.81, 3...",POINT (378852.408 407559.6),E08000002,Bury,None,379658,410767,-2.30880,53.5931,9b925fe0-4d83-431f-83c3-434ba20a3183,"POLYGON ((382118.504 413135.864, 383878.498 41..."
4,Housing,Bury,HL/2127/00,"POLYGON ((379113.7 410329.59, 379114.66 410328...",POINT (379117.39 410313.26),E08000002,Bury,None,379658,410767,-2.30880,53.5931,9b925fe0-4d83-431f-83c3-434ba20a3183,"POLYGON ((382118.504 413135.864, 383878.498 41..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4888,Site of Biological Importance,Tameside,Mill Race & Pasture at Haughton Dale,"MULTIPOLYGON (((393687.128 393384.895, 393682....",POINT (393841.341 393512.227),E08000008,Tameside,None,394987,397995,-2.07700,53.4787,d32aeedd-2000-4d80-b729-7484ebf85e64,"POLYGON ((402525.503 401459.003, 400905.596 39..."
4889,Site of Biological Importance,Wigan,Three Sisters,"POLYGON ((357928.939 401185.293, 357957.618 40...",POINT (358367.769 401012.636),E08000010,Wigan,None,362139,402126,-2.57242,53.5145,80163100-345f-4a19-9eae-09f2bda28a68,"POLYGON ((358668.063 410971.684, 361851.801 40..."
4890,Site of Biological Importance,Manchester,Nan Nook Wood,"POLYGON ((381158.403 390125.048, 381156.853 39...",POINT (380888.856 390037.976),E08000003,Manchester,None,384591,397063,-2.23360,53.4701,6a898ce9-19ca-4131-bdb4-6d5d85051598,"POLYGON ((384375.197 404714.801, 385603.299 40..."
4891,Site of Biological Importance,Manchester,Big Wood,"MULTIPOLYGON (((383512.797 386267.952, 383512....",POINT (383309.435 386406.658),E08000003,Manchester,None,384591,397063,-2.23360,53.4701,6a898ce9-19ca-4131-bdb4-6d5d85051598,"POLYGON ((384375.197 404714.801, 385603.299 40..."


In [28]:
joined_one = joined_one.drop(columns=["LAD24NM","LAD24NMW","BNG_E","BNG_N","LONG","LAT","GlobalID"])

In [29]:
joined_one


,Category,LAName,SiteRef,geometry_x,centroid,LAD24CD,geometry_y
0,Housing,Bury,HL/0414/00,"POLYGON ((379112.8 410310.51, 379117.56 410306...",POINT (379114.012 410213.116),E08000002,"POLYGON ((382118.504 413135.864, 383878.498 41..."
1,Housing,Bury,HL/1172/00,"POLYGON ((377973.95 414475.81, 377979.24 41446...",POINT (377947.835 414384.462),E08000002,"POLYGON ((382118.504 413135.864, 383878.498 41..."
2,Housing,Bury,HL/1981/00,"POLYGON ((380839 413359.78, 380838.6 413361.95...",POINT (380885.718 413390.494),E08000002,"POLYGON ((382118.504 413135.864, 383878.498 41..."
3,Housing,Bury,HL/2004/00,"POLYGON ((378834 407567.1, 378840 407566.81, 3...",POINT (378852.408 407559.6),E08000002,"POLYGON ((382118.504 413135.864, 383878.498 41..."
4,Housing,Bury,HL/2127/00,"POLYGON ((379113.7 410329.59, 379114.66 410328...",POINT (379117.39 410313.26),E08000002,"POLYGON ((382118.504 413135.864, 383878.498 41..."
...,...,...,...,...,...,...,...
4888,Site of Biological Importance,Tameside,Mill Race & Pasture at Haughton Dale,"MULTIPOLYGON (((393687.128 393384.895, 393682....",POINT (393841.341 393512.227),E08000008,"POLYGON ((402525.503 401459.003, 400905.596 39..."
4889,Site of Biological Importance,Wigan,Three Sisters,"POLYGON ((357928.939 401185.293, 357957.618 40...",POINT (358367.769 401012.636),E08000010,"POLYGON ((358668.063 410971.684, 361851.801 40..."
4890,Site of Biological Importance,Manchester,Nan Nook Wood,"POLYGON ((381158.403 390125.048, 381156.853 39...",POINT (380888.856 390037.976),E08000003,"POLYGON ((384375.197 404714.801, 385603.299 40..."
4891,Site of Biological Importance,Manchester,Big Wood,"MULTIPOLYGON (((383512.797 386267.952, 383512....",POINT (383309.435 386406.658),E08000003,"POLYGON ((384375.197 404714.801, 385603.299 40..."


In [30]:
joined_one.rename(columns={'geometry_x': 'SBI_Location', 'geometry_y': 'Local Authority'}, inplace=True)


In [31]:
joined_one


,Category,LAName,SiteRef,SBI_Location,centroid,LAD24CD,Local Authority
0,Housing,Bury,HL/0414/00,"POLYGON ((379112.8 410310.51, 379117.56 410306...",POINT (379114.012 410213.116),E08000002,"POLYGON ((382118.504 413135.864, 383878.498 41..."
1,Housing,Bury,HL/1172/00,"POLYGON ((377973.95 414475.81, 377979.24 41446...",POINT (377947.835 414384.462),E08000002,"POLYGON ((382118.504 413135.864, 383878.498 41..."
2,Housing,Bury,HL/1981/00,"POLYGON ((380839 413359.78, 380838.6 413361.95...",POINT (380885.718 413390.494),E08000002,"POLYGON ((382118.504 413135.864, 383878.498 41..."
3,Housing,Bury,HL/2004/00,"POLYGON ((378834 407567.1, 378840 407566.81, 3...",POINT (378852.408 407559.6),E08000002,"POLYGON ((382118.504 413135.864, 383878.498 41..."
4,Housing,Bury,HL/2127/00,"POLYGON ((379113.7 410329.59, 379114.66 410328...",POINT (379117.39 410313.26),E08000002,"POLYGON ((382118.504 413135.864, 383878.498 41..."
...,...,...,...,...,...,...,...
4888,Site of Biological Importance,Tameside,Mill Race & Pasture at Haughton Dale,"MULTIPOLYGON (((393687.128 393384.895, 393682....",POINT (393841.341 393512.227),E08000008,"POLYGON ((402525.503 401459.003, 400905.596 39..."
4889,Site of Biological Importance,Wigan,Three Sisters,"POLYGON ((357928.939 401185.293, 357957.618 40...",POINT (358367.769 401012.636),E08000010,"POLYGON ((358668.063 410971.684, 361851.801 40..."
4890,Site of Biological Importance,Manchester,Nan Nook Wood,"POLYGON ((381158.403 390125.048, 381156.853 39...",POINT (380888.856 390037.976),E08000003,"POLYGON ((384375.197 404714.801, 385603.299 40..."
4891,Site of Biological Importance,Manchester,Big Wood,"MULTIPOLYGON (((383512.797 386267.952, 383512....",POINT (383309.435 386406.658),E08000003,"POLYGON ((384375.197 404714.801, 385603.299 40..."
